# Define a single test

See the folder *example*:

In [1]:
EXAMPLES_FOLDER = "../examples"

... where you can find the following files:

In [2]:
import os, pprint
[f for f in os.listdir(EXAMPLES_FOLDER) if not f.startswith('.')]

['change_case',
 'fluxomics_stationary',
 'multivariate',
 'sacurine',
 'workflow-test-suite-full.yml',
 'workflow-test-suite-min.yml',
 'workflow-test-suite.yml',
 'workflows.json']

Consider the **definition file** ``workflow-test-suite-min.yml`` (steps[3-4]), which contains the two workflow tests named ``change_case`` and ``multivariate`` respectively:

In [3]:
suite_conf_filename = os.path.join(EXAMPLES_FOLDER, "workflow-test-suite-min.yml")

In [4]:
import json, yaml
with open(suite_conf_filename, "r") as fp:
    data = yaml.load(fp)
    print(json.dumps(data, indent=4))

{
    "enable_logger": false, 
    "workflows": {
        "change_case": {
            "expected": {
                "OutputText": "change_case/expected_output"
            }, 
            "inputs": {
                "InputText": "change_case/input"
            }, 
            "file": "change_case/workflow.ga"
        }, 
        "multivariate": {
            "expected": {
                "variableMetadata_out": "multivariate/variableMetadata_out", 
                "sampleMetadata_out": "multivariate/sampleMetadata_out"
            }, 
            "inputs": {
                "DataMatrix": "multivariate/dataMatrix.tsv", 
                "SampleMetadata": "multivariate/sampleMetadata.tsv", 
                "VariableMetadata": "multivariate/variableMetadata.tsv"
            }, 
            "params": {
                "3": {
                    "predI": "1", 
                    "respC": "gender", 
                    "orthoI": "NA", 
                    "testL": "FALSE"
                }


... and focus on the first example ``change_case``:

<center>![wf-changecase-workflow](../images/ChangeCaseWorkflow.png "Workflow ChangeCase")</center>
<center>**Figure 1.** Workflow *"ChangeCase"*</center>

As Fig. 1 shows, the workflow has one input and one output. To test it, both the input and the output must be uniquely identified. Typically, Galaxy identifies them using **Names** and **Labels**, respectively.  For our sample workflow, the identifiers are:

* Name **"InputText"** for the input (Fig. 2);
* Label **"OutputText"** for the output (Fig. 3).

<div>
<div style="display: inline-block; width: 50%; margin: 0 auto;">
<center>![wf-changecase-inputdataset](../images/ChangeCaseInputDataset.png "Workflow “Input Dataset“")</center>
<center>**Figure 2.** Workflow input: *"Input Dataset"*</center>
</div>
<div style="display: inline-block">
<center>![wf-changecase-outputdataset](../images/ChangeCaseOutputDataset.png "Workflow “Output Dataset“")</center>
<center>**Figure 3.** Workflow output: *"output1"*</center>
</div>
</div>

You can programmatically inpect the Galaxy workflow definition by means of the ``Workflow`` class, which allows you to see which are the workflow inputs and outputs (steps[5-8]):

In [5]:
from wft4galaxy.wrapper import Workflow

In [6]:
wf = Workflow.load(os.path.join(EXAMPLES_FOLDER, "change_case/workflow.ga"))

In [7]:
wf.show_inputs()

-  InputText 


In [8]:
 wf.show_outputs()

'1': OutputText


Steps 9-13 define a ``WorkflowTestCase`` instance programmatically:

In [9]:
from wft4galaxy.core import WorkflowTestCase

In [10]:
# prepare the workflow definition file name
base_path = os.path.join(EXAMPLES_FOLDER, "change_case")
workflow_filename = "workflow.ga"

In [11]:
# prepare test inputs
inputs = {"InputText": {"file": "input"}}

In [12]:
# prepare test outputs
expected_outputs = {"OutputText": {"file": "expected_output"}}

In [13]:
# create test case instance
test_case = WorkflowTestCase(base_path=base_path, 
                             workflow_filename=workflow_filename, 
                             inputs=inputs, expected_outputs=expected_outputs)

You can now run your test case and inspect the results of its execution (steps [14-16]), as described in "[Run a single test](3_run_single_workflow_test.html)":

In [14]:
test_result = test_case.run(enable_logger=True)

2017-03-30 14:53:38,244 [wft4galaxy] [ INFO]  Create a history '_WorkflowTestHistory_e46ecb26-1547-11e7-8812-a45e60c4fc6b' (id: u'795b4cb2a2107ba7')
2017-03-30 14:53:40,120 [wft4galaxy] [ INFO]  Workflow '_WorkflowTest_Change Case (imported from API)' (id: 795b4cb2a2107ba7) running ...
2017-03-30 14:53:45,298 [wft4galaxy] [ INFO]  waiting for datasets
2017-03-30 14:53:45,690 [wft4galaxy] [ INFO]  44015a2f24f1f430: queued
2017-03-30 14:53:46,639 [wft4galaxy] [ INFO]  44015a2f24f1f430: queued
2017-03-30 14:53:47,330 [wft4galaxy] [ INFO]  44015a2f24f1f430: running
2017-03-30 14:53:48,075 [wft4galaxy] [ INFO]  44015a2f24f1f430: running
2017-03-30 14:53:49,025 [wft4galaxy] [ INFO]  44015a2f24f1f430: running
2017-03-30 14:53:49,660 [wft4galaxy] [ INFO]  44015a2f24f1f430: ok
2017-03-30 14:53:50,165 [wft4galaxy] [ INFO]  Workflow '_WorkflowTest_Change Case (imported from API)' (id: 795b4cb2a2107ba7) executed
2017-03-30 14:53:50,168 [wft4galaxy] [ INFO]  Checking test output: ...
2017-03-30 14:

In [15]:
print("Test %s:\n\t - workflow: [%s] \n\t - results: %r" % \
      (test_result.test_id, test_result.workflow.name, test_result.results))

Test e46ecb26-1547-11e7-8812-a45e60c4fc6b:
	 - workflow: [_WorkflowTest_Change Case (imported from API)] 
	 - results: {u'OutputText': True}


In [16]:
test_result.check_output("OutputText")

True